In [ ]:
# -*- coding: utf-8 -*-

"""
{Description}

MIT License

Copyright (c) 2021, Fantasy-Sidelines

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY +OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

Sportradar API details and documentation: https://developer.sportradar.com/docs
MySportsFeeds API details and documentation: https://www.mysportsfeeds.com/data-feeds/api-docs/
www.pro-football-reference.come and documentation: https://www.sports-reference.com/termsofuse.html
"""

import time
import pyodbc
import os

# import pandas as pd
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy.engine import URL
from dotenv import load_dotenv

__author__ = "Tim Cuddeback"
__copyright__ = "Copyright (c) 2021, Fantasy-Sidelines"
__credits__ = ["Tim Cuddeback", "Sportradar API", "Fantasy Sharks", "MySportsFeeds API"]
__license__ = "MIT License"
__version__ = "1.1.0"
__maintainer__ = "Tim Cuddeback"
__email__ = "cuddebtj@gmail.com"
__status__ = "Dev"

from sql_upload import *

In [ ]:
load_dotenv()
sql_driver = os.getenv("sql_driver")
# check IP address for connection timeout errors
sql_server_16 = os.getenv("sql_server_16")
sql_server_17 = os.getenv("sql_server_17")
sql_database = os.getenv("sql_database")
sql_username = os.getenv("sql_username")
sql_password = os.getenv("sql_password")
api_key = os.getenv("sportradar_apikey")

year = [2020, 2019, 2018, 2017, 2016, 2015]

connection_string = (
    "DRIVER="
    + sql_driver
    + ";SERVER="
    + sql_server_16
    + ";DATABASE="
    + sql_database
    + ";UID="
    + sql_username
    + ";PWD="
    + sql_password
)
# cxn = pyodbc.connect(connection_string)
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)
conn = engine.connect()

inspector = inspect(engine)
print(inspector.get_table_names())

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 15)
pd.set_option("display.max_rows", None)
# schedule_stats_api_sql(api_key, year, engine)
# snaps(2016, 2020, engine)
# injuries(y1, y2, engine)
# practice_participation(season_start, season_end, engine)
# player_table(engine)
# game_table(engine)
# season_table(engine)
# week_table(engine)
# team_table(engine)
# venue_table(engine)
# calendar_table("8/1/2016", "2/1/2021", engine)
# weekly_stats_offense(conn, engine)

# calendar_ID = pd.read_sql_table("IDCalendarTable", con=conn)
# game_ID = pd.read_sql_table("IDGameTable", con=conn)
# team_ID = pd.read_sql_table("IDTeamTable", con=conn)
# venue_ID = pd.read_sql_table("IDVenueTable", con=conn)
# week_ID = pd.read_sql_table("IDWeekTable", con=conn)
# player_practice = pd.read_sql_table("playerPractice", con=conn)
# player_snaps = pd.read_sql_table("playerSnaps",  con=conn)
# player_stats = pd.read_sql_table("playerStats", con=conn)
# schedule = pd.read_sql_table("schedule", con=conn)
# team_stats = pd.read_sql_table("teamStats", con=conn)
weekly_stats = pd.read_sql_table("weeklyStats", con=conn)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

%matplotlib inline
weekly_stats.sort_values(
    ["season.year", "week.sequence", "team.id", "player.id"], inplace=True
)

In [ ]:
variables = ["off.snaps", "total_fan_pts.half.kick_yrds"]

position = ["QB", "RB", "WR", "TE"]

status_cats = ["date1.status", "date2.status", "date3.status", "game.status"]

injury_cats = [
    "head",
    "face",
    "neck",
    "shoulder",
    "upper_arm",
    "elbow",
    "forearm",
    "wrist",
    "hand_finger",
    "thumb",
    "back",
    "chest",
    "abdomen",
    "hip",
    "groin",
    "quadricep",
    "hamstring",
    "thigh",
    "knee",
    "lower_leg",
    "achilles",
    "ankle",
    "foot",
    "toe",
    "illness",
]

alpha = 0.05

hypothesis_results = {
    2: "Reject the null: There is significance between injury status/injury and fantasy production.".upper(),
    1: "Reject the null: There is significance between injury status/injury and offense snap counts.".upper(),
}

In [ ]:
hypothesis_1 = weekly_stats[
    (weekly_stats["played"] >= 1) & (weekly_stats["off.snaps"] >= 15)
]

In [ ]:
for pos in position:
    for status_day in status_cats:
        for injury in injury_cats:

            hypothesis_1_full = hypothesis_1[
                (
                    (hypothesis_1[status_day].isnull())
                    | (hypothesis_1[status_day] == "Full")
                )
                & (hypothesis_1["player.position"] == pos)
            ]

            hypothesis_1_status = hypothesis_1[
                ~(
                    (hypothesis_1[status_day].isnull())
                    | (hypothesis_1[status_day] == "Full")
                )
                & (hypothesis_1["player.position"] == pos)
                & (hypothesis_1[injury] > 0)
            ]

            try:
                p_mannwhiteyu_snaps = stats.mannwhitneyu(
                    hypothesis_1_full[variables[0]], hypothesis_1_status[variables[0]]
                )[1]
                if p_mannwhiteyu_snaps > alpha:
                    pass
                elif p_mannwhiteyu_snaps <= alpha:
                    print(
                        pos,
                        injury.upper(),
                        status_day.upper(),
                        "Snaps",
                        hypothesis_results[1],
                        "p_value = " + str(p_mannwhiteyu_snaps),
                        "No designation Snaps:",
                        len(hypothesis_1_full[variables[0]]),
                        "Designation Snaps:",
                        len(hypothesis_1_status[variables[0]]),
                        "\n",
                        sep="\n",
                    )

                p_mannwhiteyu_fp = stats.mannwhitneyu(
                    hypothesis_1_full[variables[1]], hypothesis_1_status[variables[1]]
                )[1]
                if p_mannwhiteyu_fp > alpha:
                    pass
                elif p_mannwhiteyu_fp <= alpha:
                    print(
                        pos,
                        injury.upper(),
                        status_day.upper(),
                        "FP",
                        hypothesis_results[2],
                        "p_value = " + str(p_mannwhiteyu_fp),
                        "No designation FP:",
                        len(hypothesis_1_full[variables[1]]),
                        "Designation FP:",
                        len(hypothesis_1_status[variables[1]]),
                        "\n",
                        sep="\n",
                    )
            except:
                continue

In [ ]:
for status_day in status_cats:

    hypothesis_1_full = hypothesis_1[
        ((hypothesis_1[status_day].isnull()) | (hypothesis_1[status_day] == "Full"))
    ]

    hypothesis_1_status = hypothesis_1[
        # (
        #     (hypothesis_1[status_day] == "Questionable")
        #     | (hypothesis_1[status_day] == "Doubtful")
        #     | (hypothesis_1[status_day] == "Limited")
        #     | (hypothesis_1[status_day] == "DNP")
        # )
        ~((hypothesis_1[status_day].isnull()) | (hypothesis_1[status_day] == "Full"))
    ]

    try:
        p_mannwhiteyu_snaps = stats.mannwhitneyu(
            hypothesis_1_full[variables[0]], hypothesis_1_status[variables[0]]
        )[1]
        if p_mannwhiteyu_snaps > alpha:
            pass
        elif p_mannwhiteyu_snaps <= alpha:
            print(
                status_day.upper(),
                "Snaps",
                hypothesis_results[1],
                "p_value = " + str(p_mannwhiteyu_snaps),
                "No designation Snaps:",
                len(hypothesis_1_full[variables[0]]),
                "Designation Snaps:",
                len(hypothesis_1_status[variables[0]]),
                "\n",
                sep="\n",
            )

        p_mannwhiteyu_fp = stats.mannwhitneyu(
            hypothesis_1_full[variables[1]], hypothesis_1_status[variables[1]]
        )[1]
        if p_mannwhiteyu_fp > alpha:
            pass
        elif p_mannwhiteyu_fp <= alpha:
            print(
                status_day.upper(),
                "FP",
                hypothesis_results[2],
                "p_value = " + str(p_mannwhiteyu_fp),
                "No designation FP:",
                len(hypothesis_1_full[variables[1]]),
                "Designation FP:",
                len(hypothesis_1_status[variables[1]]),
                "\n",
                sep="\n",
            )
    except:
        continue

In [ ]:
"""
paired t-test
    assume normal distribution
    assume no outliers

Wilcoxon Signed-Rank test:
    not normal distribution
"""

In [ ]:
conn.close()
# cxn.close()

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# from scipy import stats
# %matplotlib inline
# weekly_stats = pd.read_sql_table("weeklyStats", con=conn)
# mask = (weekly_stats["player.position"] == "RB") & (weekly_stats["total.snaps"] >= 5) & (weekly_stats["total_fan_pts.half.kick_yrds"] >= 0)
# sns.displot(weekly_stats[mask], x="total_fan_pts.half.kick_yrds", binwidth=1, kde=True)
# corr = weekly_stats.corr()
# def background_pandas(x):
#     try:
#         if x >= 0.5:
#             bg_color = "green"
#         elif x <= -0.5:
#             bg_color = "green"
#         else:
#             bg_color = ""
#         return f"background-color : {bg_color}"

#     except:
#         return "background-color: ''"

# corr.style.format(precision=3).applymap(background_pandas).to_excel("correlations.xlsx", engine="openpyxl")